In [1]:
!python -m spacy download ru_core_news_sm
!pip install razdel
!pip install gensim
!pip install catboost
!pip install pymorphy2

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 68.5 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for pymorphy3>=1.0.0 from https://files.pythonhosted.org/packages/d7/f9/ffb9afde503dc6bb2361ea79ceaea18138fbcee32aec4c5d8efa49180753/pymorphy3-1.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for docopt-ng>=0.6 from https://files.pythonhosted.org/packages/6c/4a/c3b77fc1a24510b08918b43a473410c0168f6e657118807015f1f1edceea/docopt_ng-0.9.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via sp

In [2]:
import pandas as pd
import numpy as np
import re

import spacy
import gensim.models

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import razdel
import pymorphy2

import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt')
nltk.download('stopwords')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# функция, возвращает векторизированное предложение
def get_vector(model, sentence, vector_size=100):
    sentence_vector = []

    if len(sentence) == 0:
        # Пустые предложения заполним их одним  словом
        token_vector = np.zeros(vector_size)
        sentence_vector.append(token_vector)
    else:
        for token in sentence:
            try:
                token_vector = model.wv[token]
            except KeyError as e:
                print('here')
                # Случай неизвестного слова
                token_vector = np.zeros(vector_size)
            finally:
                sentence_vector.append(token_vector)
    
    return np.mean(sentence_vector, axis=0)


# функция, возвращает векторизированные выборки
def vectorize_train_test(model, X_train, X_test, vector_size=100):
    X_train_vectorized = np.zeros((X_train.shape[0], vector_size))
    for index, sentence in enumerate(X_train):
        X_train_vectorized[index] = get_vector(model, sentence, vector_size)

    X_test_vectorized = np.zeros((X_test.shape[0], vector_size))
    for index, sentence in enumerate(X_test):
        X_test_vectorized[index] = get_vector(model, sentence,  vector_size)

    return X_train_vectorized, X_test_vectorized


# функция, очистка от html разметки
def clean_html_bs4(text_data):
    soup = BeautifulSoup(text_data, 'lxml')
    return soup.get_text()

# функция, очистка от мусора, нормализация и лемматизация
def tokenize(text, stopwords, need_lemmatize=False):
    result = []
    sentences = [item.text for item in razdel.sentenize(str(text))]

    for sentence in sentences:
        text = sentence.lower()
        text = clean_html_bs4(text)
        text = re.sub(r"\s+", ' ', text)

        tokens = [item.text for item in  razdel.tokenize(text)]
        tokens = [re.sub("[^а-яА-Яa-zA-Z]", ' ', item) for item in tokens]

        if need_lemmatize:
            tokens = [analyzer.parse(token)[0].normal_form for token in tokens if token not in stopwords  and ' ' not in token and len(token) > 2]
            tokens = [token for token in tokens if token not in lemmatized_sw]
        tokens = [re.sub(r"ё", "е", token) for token in tokens]
        result.extend(tokens)
    return result

## Загрузка датасета

In [5]:
df = pd.read_csv('/kaggle/input/vacancies/data_vacancies.csv')
df.head(5)

,id,custom_position,schedule,salary_from,salary_to,salary_pay_type,offer_education_id,education_name,education_is_base,education_order_num,city_id,list_regions,work_skills,tags_id
0,48202096,Сварщик-сборщик,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['сварочные работы', 'сборка изделий по чертеж...",NaN
1,48202097,Сварщик-монтажник,полный рабочий день,60000,120000,NaN,0,любое,True,0,2,[4],"['монтажные работы', 'строительные работы', 'э...",NaN
2,48202098,Слесарь-сборщик,полный рабочий день,60000,80000,NaN,0,любое,True,0,2,[4],"['работа на фрезерных станках', 'слесарный рем...",NaN
3,48202356,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,1,[3],"['комплектация товара', 'маркировка', 'стрессо...","[6, 9]"
4,48202357,Грузчик-упаковщик,частичная занятость,30000,35000,NaN,0,любое,True,0,57,"[181, 182, 183, 185, 186, 187, 188, 189, 190, ...","['маркировка', 'стрессоустойчивость', 'погрузо...","[6, 9]"


In [6]:
df.shape

(19489, 14)

#### Выделение полей

In [7]:
columns = ['custom_position', 'schedule', 'salary_to', 'city_id', 'work_skills']
df = df[columns]
df.head(5)

,custom_position,schedule,salary_to,city_id,work_skills
0,Сварщик-сборщик,полный рабочий день,120000,2,"['сварочные работы', 'сборка изделий по чертеж..."
1,Сварщик-монтажник,полный рабочий день,120000,2,"['монтажные работы', 'строительные работы', 'э..."
2,Слесарь-сборщик,полный рабочий день,80000,2,"['работа на фрезерных станках', 'слесарный рем..."
3,Грузчик-упаковщик,частичная занятость,35000,1,"['комплектация товара', 'маркировка', 'стрессо..."
4,Грузчик-упаковщик,частичная занятость,35000,57,"['маркировка', 'стрессоустойчивость', 'погрузо..."


#### Возьмем вакансии для топ5 локаций

In [8]:
top5_loc = df['city_id'].value_counts()[:5].index.tolist()
df = df[df['city_id'].isin(top5_loc)]
df.head(5)

,custom_position,schedule,salary_to,city_id,work_skills
0,Сварщик-сборщик,полный рабочий день,120000,2,"['сварочные работы', 'сборка изделий по чертеж..."
1,Сварщик-монтажник,полный рабочий день,120000,2,"['монтажные работы', 'строительные работы', 'э..."
2,Слесарь-сборщик,полный рабочий день,80000,2,"['работа на фрезерных станках', 'слесарный рем..."
3,Грузчик-упаковщик,частичная занятость,35000,1,"['комплектация товара', 'маркировка', 'стрессо..."
4,Грузчик-упаковщик,частичная занятость,35000,57,"['маркировка', 'стрессоустойчивость', 'погрузо..."


#### Проверим пропуски

In [9]:
df.isna().sum()

custom_position    0
schedule           0
salary_to          0
city_id            0
work_skills        0
dtype: int64

In [10]:
df.dropna(inplace=True)

In [11]:
df.shape

(18415, 5)

### Обработка полей

#### OneHotEncoding полей city_id и schedule

In [12]:
city_id_one = pd.get_dummies(df['city_id'], prefix='city_id')
schedule_one = pd.get_dummies(df['schedule'], prefix='schedule')

df = pd.concat([df, city_id_one], axis=1)
df = pd.concat([df, schedule_one], axis=1)

df.drop(columns=['city_id'], inplace=True)
df.drop(columns=['schedule'], inplace=True)
df.head(5)

,custom_position,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость
0,Сварщик-сборщик,120000,"['сварочные работы', 'сборка изделий по чертеж...",False,True,False,False,False,False,True,False,False,False,False
1,Сварщик-монтажник,120000,"['монтажные работы', 'строительные работы', 'э...",False,True,False,False,False,False,True,False,False,False,False
2,Слесарь-сборщик,80000,"['работа на фрезерных станках', 'слесарный рем...",False,True,False,False,False,False,True,False,False,False,False
3,Грузчик-упаковщик,35000,"['комплектация товара', 'маркировка', 'стрессо...",True,False,False,False,False,False,False,False,False,False,True
4,Грузчик-упаковщик,35000,"['маркировка', 'стрессоустойчивость', 'погрузо...",False,False,True,False,False,False,False,False,False,False,True


#### Нормализация и токенизации поля custom_position

In [13]:
nlp = spacy.load('ru_core_news_sm')

df['custom_position_clear'] = df['custom_position'].apply(
    lambda x: ' '.join([str(token).lower() for token in nlp(x) if
      not token.is_stop
      and not token.is_punct
      and not token.is_digit
      and not token.like_email
      and not token.like_num
      and not token.is_space
    ])
  )

df.drop(columns=['custom_position'], inplace=True)
df.head(5)

,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear
0,120000,"['сварочные работы', 'сборка изделий по чертеж...",False,True,False,False,False,False,True,False,False,False,False,сварщик сборщик
1,120000,"['монтажные работы', 'строительные работы', 'э...",False,True,False,False,False,False,True,False,False,False,False,сварщик монтажник
2,80000,"['работа на фрезерных станках', 'слесарный рем...",False,True,False,False,False,False,True,False,False,False,False,слесарь сборщик
3,35000,"['комплектация товара', 'маркировка', 'стрессо...",True,False,False,False,False,False,False,False,False,False,True,грузчик упаковщик
4,35000,"['маркировка', 'стрессоустойчивость', 'погрузо...",False,False,True,False,False,False,False,False,False,False,True,грузчик упаковщик


#### Обработка work_skills

In [14]:
for index, row in df.iterrows():
    df['work_skills'][index] = ', '.join([word.replace('\'', '').lower() 
                                         for word in row['work_skills'].split('[')[1].split(']')[0].split(', ')])
df.head(5)

,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear
0,120000,"сварочные работы, сборка изделий по чертежам, ...",False,True,False,False,False,False,True,False,False,False,False,сварщик сборщик
1,120000,"монтажные работы, строительные работы, электро...",False,True,False,False,False,False,True,False,False,False,False,сварщик монтажник
2,80000,"работа на фрезерных станках, слесарный ремонт,...",False,True,False,False,False,False,True,False,False,False,False,слесарь сборщик
3,35000,"комплектация товара, маркировка, стрессоустойч...",True,False,False,False,False,False,False,False,False,False,True,грузчик упаковщик
4,35000,"маркировка, стрессоустойчивость, погрузочно-ра...",False,False,True,False,False,False,False,False,False,False,True,грузчик упаковщик


#### Дедубликация

In [15]:
columns = list(df.columns)
columns.remove('salary_to')
columns

['work_skills',
 'city_id_1',
 'city_id_2',
 'city_id_57',
 'city_id_102',
 'city_id_174',
 'schedule_вахта',
 'schedule_полный рабочий день',
 'schedule_свободный график',
 'schedule_сменный график',
 'schedule_удаленная работа',
 'schedule_частичная занятость',
 'custom_position_clear']

In [16]:
df_duplicated = df[df[columns].duplicated(keep=False)]
df_duplicated.shape

(2087, 14)

In [17]:
df = df.drop(df[df[columns].duplicated(keep=False)].index)
df.head(5)

,salary_to,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear
0,120000,"сварочные работы, сборка изделий по чертежам, ...",False,True,False,False,False,False,True,False,False,False,False,сварщик сборщик
1,120000,"монтажные работы, строительные работы, электро...",False,True,False,False,False,False,True,False,False,False,False,сварщик монтажник
2,80000,"работа на фрезерных станках, слесарный ремонт,...",False,True,False,False,False,False,True,False,False,False,False,слесарь сборщик
3,35000,"комплектация товара, маркировка, стрессоустойч...",True,False,False,False,False,False,False,False,False,False,True,грузчик упаковщик
4,35000,"маркировка, стрессоустойчивость, погрузочно-ра...",False,False,True,False,False,False,False,False,False,False,True,грузчик упаковщик


In [18]:
df_deduplicated = pd.DataFrame(df_duplicated.groupby(columns)['salary_to'].median()).reset_index()
df_deduplicated.shape

(673, 14)

In [19]:
df = pd.concat([df.drop(['salary_to'], axis=1), df['salary_to']], axis=1)
df = pd.concat([df, df_deduplicated], axis=0).reset_index(drop=True)

In [20]:
df

,work_skills,city_id_1,city_id_2,city_id_57,city_id_102,city_id_174,schedule_вахта,schedule_полный рабочий день,schedule_свободный график,schedule_сменный график,schedule_удаленная работа,schedule_частичная занятость,custom_position_clear,salary_to
0,"сварочные работы, сборка изделий по чертежам, ...",False,True,False,False,False,False,True,False,False,False,False,сварщик сборщик,120000.0
1,"монтажные работы, строительные работы, электро...",False,True,False,False,False,False,True,False,False,False,False,сварщик монтажник,120000.0
2,"работа на фрезерных станках, слесарный ремонт,...",False,True,False,False,False,False,True,False,False,False,False,слесарь сборщик,80000.0
3,"комплектация товара, маркировка, стрессоустойч...",True,False,False,False,False,False,False,False,False,False,True,грузчик упаковщик,35000.0
4,"маркировка, стрессоустойчивость, погрузочно-ра...",False,False,True,False,False,False,False,False,False,False,True,грузчик упаковщик,35000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16996,электромонтажные работы,False,False,True,False,False,False,True,False,False,False,False,электромонтажник,100000.0
16997,электромонтажные работы,False,True,False,False,False,False,True,False,False,False,False,электромонтажник,100000.0
16998,электромонтажные работы,True,False,False,False,False,False,True,False,False,False,False,макетчик монтажник,200000.0
16999,электромонтажные работы,True,False,False,False,False,False,True,False,False,False,False,электромонтажник,100000.0


#### Обработка custom_position

Для обработки остальных полей необходимо предварительно разделить на тренировочную и тестовую выборки

In [21]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'salary_to'], df['salary_to'], random_state=2023)

In [22]:
fasttext = gensim.models.FastText(vector_size=100, min_count=1)
fasttext.build_vocab(corpus_iterable=X_train['custom_position_clear'])

fasttext.train(
    corpus_iterable=X_train['custom_position_clear'],
    total_examples=len(X_train['custom_position_clear']),
    epochs=20
    )

(1229469, 6056040)

In [23]:
cuspos_vec_train, cuspos_vec_test = vectorize_train_test(fasttext, X_train['custom_position_clear'], X_test['custom_position_clear'])
cuspos_vec_train = pd.DataFrame(cuspos_vec_train).add_prefix('custom_position_')
cuspos_vec_test = pd.DataFrame(cuspos_vec_test).add_prefix('custom_position_')

In [24]:
X_train = pd.concat([X_train.reset_index(drop=True), cuspos_vec_train], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), cuspos_vec_test], axis=1)

X_train.drop(columns=['custom_position_clear'], inplace=True)
X_test.drop(columns=['custom_position_clear'], inplace=True)

In [25]:
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

#### Обработка work_skills

In [26]:
sw = stopwords.words('russian')
additional_sw =  'мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы '
pronouns = 'я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий'
conjunctions = 'что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда'
digits = 'ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто'
modal_words = 'вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется'
particles = 'да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли'
prepositions = 'близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно'
evaluative = 'наиболее наименее лучший больший высший низший худший более менее'

sw.extend(additional_sw.split())
sw.extend(pronouns.split())
sw.extend(conjunctions.split())
sw.extend(digits.split())
sw.extend(modal_words.split())
sw.extend(particles.split())
sw.extend(prepositions.split())
sw.extend(evaluative.split())
sw = list(set(sw))

In [27]:
need_preprocess = True

if need_preprocess:
    analyzer = pymorphy2.MorphAnalyzer()
    lemmatized_sw = [analyzer.parse(word)[0].normal_form for word in sw]
    
    for index, row in X_train.iterrows():
        X_train['work_skills'][index] = tokenize(row['work_skills'], stopwords=sw, need_lemmatize=True)
    for index, row in X_test.iterrows():
        X_test['work_skills'][index] = tokenize(row['work_skills'], stopwords=sw, need_lemmatize=True)

In [28]:
for index, row in X_train.iterrows():
    X_train['work_skills'][index] = " ".join(row['work_skills'])
    
for index, row in X_test.iterrows():
    X_test['work_skills'][index] = " ".join(row['work_skills'])

In [29]:
vectorizer = TfidfVectorizer(min_df=0.003, token_pattern=r'(?u)\b\w\w+\b')
ws_vec_train = vectorizer.fit_transform(X_train['work_skills'])
ws_vec_test = vectorizer.transform(X_test['work_skills'])

ws_vec_train = pd.DataFrame(ws_vec_train.toarray(), columns=vectorizer.get_feature_names_out()).add_prefix('ws_')
ws_vec_test = pd.DataFrame(ws_vec_test.toarray(), columns=vectorizer.get_feature_names_out()).add_prefix('ws_')

X_train = pd.concat([X_train.reset_index(drop=True), ws_vec_train], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), ws_vec_test], axis=1)

X_train.drop(columns=['work_skills'], inplace=True)
X_test.drop(columns=['work_skills'], inplace=True)

#### Обучение модели

In [35]:
ridge_params = {'alpha': [0.1, 0.5, 1], 
                'solver': ['svd', 'sparse_cg', 'lsqr', 'sag']}
lasso_params = {'alpha': [0.1, 0.5, 1], 
                'selection': ['cyclic', 'random'],
                'max_iter': [100, 500, 1000]}
elastic_params = {'alpha': [0.1, 0.5, 1], 
                  'l1_ratio': [0, 0.5, 1],
                  'selection': ['cyclic', 'random'],
                  'max_iter': [100, 500, 1000]}

In [36]:
ridge = Ridge()
# lin_reg = LinearRegression()
lasso = Lasso()
elastic = ElasticNet(random_state=0)

In [37]:
cv_ridge = GridSearchCV(ridge, ridge_params, n_jobs=-1)
cv_ridge.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


GridSearchCV(estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1],
                         'solver': ['svd', 'sparse_cg', 'lsqr', 'sag']})

In [39]:
cv_lasso = GridSearchCV(lasso, lasso_params, n_jobs=-1)
cv_lasso.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

GridSearchCV(estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1], 'max_iter': [100, 500, 1000],
                         'selection': ['cyclic', 'random']})

In [41]:
cv_elastic = GridSearchCV(elastic, elastic_params, n_jobs=-1)
cv_elastic.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isi

GridSearchCV(estimator=ElasticNet(random_state=0), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1], 'l1_ratio': [0, 0.5, 1],
                         'max_iter': [100, 500, 1000],
                         'selection': ['cyclic', 'random']})

In [42]:
print('Ridge best params: ', cv_ridge.best_params_)
print('Lasso best params: ', cv_lasso.best_params_)
print('Elastic best params: ', cv_elastic.best_params_)

Ridge best params:  {'alpha': 1, 'solver': 'svd'}
Lasso best params:  {'alpha': 1, 'max_iter': 1000, 'selection': 'random'}
Elastic best params:  {'alpha': 1, 'l1_ratio': 1, 'max_iter': 1000, 'selection': 'random'}


In [43]:
best_ridge = cv_ridge.best_estimator_
best_lasso = cv_lasso.best_estimator_
best_elastic = cv_elastic.best_estimator_

In [44]:
y_pred = best_ridge.predict(X_test)
r2_score = best_ridge.score(X_test, y_test)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("RIDGE")
print('r2_score:', r2_score)
print('mape:', mape)

RIDGE
r2_score: 0.43497977151162015
mape: 0.3144927762741055


In [45]:
best_ridge.coef_.max()

195507.3863284083

In [46]:
y_pred = best_lasso.predict(X_test)
r2_score = best_lasso.score(X_test, y_test)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("LASSO")
print('r2_score:', r2_score)
print('mape:', mape)

LASSO
r2_score: 0.4343505955435434
mape: 0.3146180169521783


In [47]:
best_lasso.coef_.max()

258591.57524557388

In [48]:
y_pred = best_elastic.predict(X_test)
r2_score = best_elastic.score(X_test, y_test)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("Elastic Net")
print('r2_score:', r2_score)
print('mape:', mape)

Elastic Net
r2_score: 0.4344002181638712
mape: 0.3145976167315034


In [49]:
best_elastic.coef_.max()

258589.06558924093

$
cs - custom\_postion \ vector \\
ws - work\_skill \ vector \\
\gamma = \cos(cs, ws) \\
$

Вместо обучения модели:
$ \\
y = const + \beta_0 \cdot cs + \beta_1 \cdot ws_1 + \beta_2 \cdot ws_2 + \beta_3 \cdot ws_3 + ...
$

Обучать: 
$ \\
y = const + \beta_0 \cdot cs + \beta_1 \cdot \gamma_1 \cdot ws_1 + \beta_2 \cdot \gamma_2 \cdot ws_2 + \beta_3 \cdot \gamma_3 \cdot ws_3 + ...
$